In [2]:
import pandas as pd
import numpy as np

## NOTE: This notebook is "under construction". Furthermore, file needed for the 2nd part not in the repository.

# Very Incomplete - Still no relevant results

Most of this is adapted directly from FTMSVisualization (https://github.com/wkew/FTMSVisualization) especially 0-FormulaGenerator.py and 1-FormulaAssignment.py files that are the ones relevant to what we want to achieve - make a "database" of possible formulas with their respective exact teoric mass and then assign to a set of m/z peaks a specific formula from said database.

Apart from this, the 2007 paper "Seven Golden Rules for heuristic filtering of molecular formulas obtained by accurate mass spectrometry" of Tobias Kind and Oliver Fiehn was used as a base for the rules used to make the "formula database".

### Warning!

I advise not to run this notebook since it takes so much time (like a day) and it also asks a lot for the pc, specifically, RAM memory. For both of this reasons, this shouldn't be ran lightly.

#### Some of the issues encountered so far with this process

First off, this is a list of peaks from MetaboScape so different peaks due to the different isotopes of the same compound should've already been merged together, right? As such, both rule nº 3 (the isotopic pattern filter) and rule nº 7 (TMS check) aren't applicable. Furthermore, this means that peak formula identification can't be consubstantiated with its isotopic distribution (thta kind of information is already lost), that is, it can't be used to help make a formula assignemnt more reliable or try to see which formula of the multiple formuals that might be assigned to one peak might be better.

However, the biggest problem of following the rules so far is rule nº 2 - Lewis and Senior check. This is the biggest problem because it is completely possible to implement, I think I undertand how to. FTMSVisualization did implement a strict rule where they checked the sum of the H and N (Na and K too, ionic compositions) elements to see if the formula...

#### Commentary not COMPLETE - situation is evolving

## Part nº 1 - Building the Formula "Database"

### Setting up functions to build the formula database

First a dictionary with all the masses and abundances of different elements (taken from FTMSVisualization).

In [3]:
#atomic masses taken from Pure Appl. Chem. 2016; 88(3): 265–291, Atomic Weights of the Elements 2013, doi: 10.1515/pac-2015-0305
#Atomic Masses taken from AME2012 - Chinese Physics C 36 (2012)  1603-2014, Wang, Audi, Wapstra, Kondex, MacCormic, Xu, and Pfeiffer. doi: 10.1088/1674-1137/36/12/003
#isotoptic abundances from Pure Appl. Chem. 2016; 88(3): 293–306, Isotopic compositions of the elements 2013 (IUPAC Technical Report), doi: 10.1515/pac-2015-0503
#electron mass from NIST http://physics.nist.gov/cgi-bin/cuu/Value?meu|search_for=electron+mass
chemdict = {'H':(1.007825, 0.99984),
            'C':(12.000000, 0.98892),
            'N':(14.003074, 0.99634),
            'O':(15.994915, 0.99762),
            'Na':(22.989769, 1.0),
            'P':(30.973763,1.0),
            'S':(31.972071, 0.95041),
            'Cl':(34.968853, 0.75765),
            'K':(38.963706, 0.93258),
            'Br':(78.918338, 0.50686)} 

Function to calculate formulas exact masses

In [4]:
def getmass(c,h,o,n,s,p,na,k):
    "Get the exact mass for any formula."
    massC = chemdict['C'][0] * c
    massH = chemdict['H'][0] * h
    massO = chemdict['O'][0] * o
    massN = chemdict['N'][0] * n
    massS = chemdict['S'][0] * s
    massP = chemdict['P'][0] * p
    massNa = chemdict['Na'][0] * na
    massK = chemdict['K'][0] * k

    massTotal = massC + massH + massO + massN + massS +massP + massP + massNa + massK

    return massTotal

Function to calculate the natural abundances of the formulas made of all the most common isotopes for each element.

In [5]:
def getabun(c,h,o,n,s):
    "The natural abundance of the formulas made of all the most common isotopes for each element."
    abunC, abunH, abunO, abunN, abunS = 1,1,1,1,1
    
    if c > 0:
        abunC = chemdict['C'][1] ** c

    if h > 0:
        abunH = chemdict['H'][1] ** h

    if o > 0:
        abunO = chemdict['O'][1] ** o

    if n > 0:
        abunN = chemdict['N'][1] ** n

    if s > 0:
        abunS = chemdict['S'][1] ** s

    abunTotal = abunC * abunH * abunO * abunN * abunS
    return abunTotal

### Maximum number of elements and ranges for element proportions according to the 7 golden rules paper

In [6]:
#if m < 500:
#    maxC,maxH,maxO,maxN,maxS,maxP = 39,72,20,20,10,9
#elif m < 1000: #36 7 golden rules?
#    maxC,maxH,maxO,maxN,maxS,maxP = 78,126,27,25,14,9
#elif m < 2000:
#    maxC,maxH,maxO,maxN,maxS,maxP = 156,180,63,32,14,9

com_range = {'H/C':(0.2,3.1),'N/C':(0,1.3),'O/C':(0,1.2),'P/C':(0,0.3),'S/C':(0,0.8)} #99.7% of all existing formulas
ext_range = {'H/C':(0.1,6),'N/C':(0,4),'O/C':(0,3),'P/C':(0,2),'S/C':(0,3)} #99.99% of all existing formulas

### Maximum number of elements and ranges for element proportions as per the chemical constraints put upon on the MetaboScape

Issues: I might add the F element easily, however adding Cl might prove a challenge due to the 75, 25% abundances of its two stable isotopes which can complicate the analysis. The first thing that comes to mind to try and solve this problem is to almost make two different "elements" whose sum can't be greater than the limit imposed. Even if it would lead to slower analysis times it could in theory work. However, I do not know how the bucket labels in MetaboScape are made, that is, if they follow a certain convention that alter the m/z peaks detected to their monoisotopic masses in the labels and if I should only consider Cl35 or not. Furthermore, doing it considering both would lead to peaks with considerable different masses to have the same formula assigned.

In [7]:
#Rules applied to the file:
maxC,maxH,maxO,maxN,maxS,maxP = 39,72,20,20,5,9
maxF, maxCl = 8,5
ms_range = {'H/C':(0.2,3.1),'N/C':(0,1.3),'O/C':(0,1.2),'P/C':(0,0.3),'S/C':(0,0.8),'F/C':(0,1.5), 'Cl/C':(0,0.8)}

### Function to give every formula possible and their exact mass between a certain mass interval and according to other limitations such as the 7 golden rules paper or specifically imposed conditions (adapted from FTMSVisualization)

Rules nº 1, 4, 5 and 6 currently followed.

Working on integrating rule nº 2 however I think it may be better to apply it on the assigning a formula part of the notebook even though the process will be slwoer this way since the 7 golden rules paper says that some formulas don't follow this rule and so it might be better to use it as a decider between formulas when multiple can be assigned to the same peak.

In [30]:
def form_calc(low, high, elem_range = com_range):
    """Calculates all formulas possible (according to some stipulations) between a certain mass interval.
    
       low: scalar; lower limit of the molecular mass of the formula.
       high: scalar, upper limit of the molecular mass of the formula.
       elem_range: dictionary; dictionary where keys are string of the ratios of certain elements (examples: 'H/C', 'O/C') and
    their values are a tuple with the minimum and maximum ratios they can have in a certain formula (rule nº5).
    
       return: dictionary where keys are exact masses and values are tuples with the overall abundance of the monoisotopic 'mass'
    (maybe superfluous) and the number of atoms of each elements (in a specific order)."""
    
    #RULE Nº 1 - Following the 7 golden rule limits
    if high <= 500:
        maxC,maxH,maxO,maxN,maxS,maxP = 39,72,20,20,10,9
    else: #ignoring the other limit since this already takes a lot of time 
    #elif high <= 1000: #36 7 golden rules?
        maxC,maxH,maxO,maxN,maxS,maxP = 78,126,27,25,14,9
    #else: #elif m < 2000:
        #maxC,maxH,maxO,maxN,maxS,maxP = 156,180,63,32,14,9
    
    maxC = min((int(high) / 12), maxC) #max carbon nº is the smaller of the total mass/12 or predefined maxC
    maxH = min((maxC * 4), maxH) #max hydrogen nº is the smaller of 4 times the nº of carbons or the predefined max hydrogen nº
    maxO = min((int(high) / 16), maxO) #max oxygen nº has to be the smaller of the total mass/16 or predefined maxO
    #Those 3 above should have a +1 next to their maxC, maxH and maxO but since I forgot on the first 
    maxN = maxN + 1
    maxS = maxS + 1
    maxP = maxP + 1

    allposs = {} #pd.DataFrame(columns = ['abundance', 'c','h','o','n','s','p'])
    
    #Infinite for's for each element and ifs that see if the each element count follow the imposed ratios in elem_range
    for c in range(int(maxC))[1:]: #molecules contain at least 1 C and 1 H
        for h in range(int(maxH))[1:]:
            hcrat = float(h)/float(c)
            if elem_range['H/C'][0] < hcrat < elem_range['H/C'][1]: #RULE Nº 4
                for p in range(maxP):
                    pcrat = float(p)/float(c)
                    if pcrat < elem_range['P/C'][1]: #RULE Nº 5
                        for o in range(int(maxO)):
                            ocrat = float(o)/float(c)
                            if ocrat < elem_range['O/C'][1]: #RULE Nº 5
                                for n in range(maxN):
                                    ncrat = float(n)/float(c)
                                    if ncrat < elem_range['N/C'][1]: #RULE Nº 5
                                        #if neg_nhchecker(h,n):
                                        for s in range(maxS):
                                            scrat = float(s)/float(c)
                                            if scrat < elem_range['S/C'][1]: #RULE Nº 5
                                                
                                                NOPS_ratio = NOPS(n,o,p,s)
                                                if NOPS_ratio: #RULE Nº 6 - element probability check - see function below
                                                    mass = getmass(c,h,o,n,s,p,0,0) #getting the exact mass of the formula
                                                    if low < mass < high: #If the mass is in the given range
                                                        #formula = "C%iH%iO%iN%iS%iP%i" % (c,h,o,n,s,p) 
                                                        abundance = getabun(c,h,o,n,s)
                                                        allposs[mass] = (abundance, c,h,o,n,s,p)
    #print(len(allposs))
    return allposs

### Rule nº 6 - HNOPS heuristic probability check

FTMSVisualization didn't impose this rule. Instead, they did something else where they said that the total number of heteroatoms had to be smaller than 1.3 times the number of carbons. I altered to this to follow the 7 golden rules paper more closely. The following function sees if there are at least 3 types of heteroatoms in the formula and then observes if each of their counts is below a threshold shown in the 7 golden rules paper. If it isn't, NOPS_ratio is False and the formula is ignored.

In [12]:
#Rule nº 6 - HNOPS heuristic probability check
def NOPS (n,o,p,s):
    """Checks if the element counts"""
    NOPS_ratio = True
    if (n > 1) and (o > 1) and (p > 1) and (s > 1): #NOPS
        if (n < 10) and (o < 20) and (p < 4) and (s < 3):
            NOPS_ratio = True
        else:
            NOPS_ratio = False
    elif (n > 3) and (o > 3) and (p > 3): #NOP
        if (n < 11) and (o < 22) and (p < 6):
            NOPS_ratio = True
        else:
            NOPS_ratio = False
    elif (o > 1) and (p > 1) and (s > 1): #OPS
        if (o < 14) and (p < 3) and (s < 3):
            NOPS_ratio = True
        else:
            NOPS_ratio = False
    elif (n > 1) and (p > 1) and (s > 1): #PSN
        if (n < 10) and (p < 4) and (s < 3):
            NOPS_ratio = True
        else:
            NOPS_ratio = False
    elif (n > 6) and (o > 6) and (s > 6): #PSN
        if (n < 19) and (o < 14) and (s < 8):
            NOPS_ratio = True
        else:
            NOPS_ratio = False

    return NOPS_ratio

Making a dictionary of dataframes with each dataframe having all possible formulas (according to the previously mentioned rules and functions) of a 100 mass interval with the name of each dataframe being dict + lower limit of said mass interval.

In [35]:
formulas = {}
for i in range(1,12):
    a = form_calc(i*100, (i+1)*100, elem_range = ms_range)
    formulas['dict' + str(i*100)] = pd.DataFrame.from_dict(a, orient = 'index', columns = ['Abundance', 'C','H','O','N','S','P'])
    formulas['dict' + str(i*100)] = formulas['dict' + str(i*100)].sort_index()
    #Writing a .csv file of the data
    formulas['dict' + str(i*100)].to_csv('dict' + str(i*100) + '.csv')
#a = form_calc(300, 300, elem_range = com_range) #using the common 99.7 common range described in the 7 golden rules paper.

## Part nº2 - Assigning Formulas to m/z peaks / bucket labels

Reading the file from MetaboScape. Due to some complications with reading the .csv files. I had to transform it into an Excel where each row was separated into columns by the "," (text to columns function of excel) adn then was read with the read_excel function from Pandas.

In [17]:
formula_file = pd.read_excel('tabela5yeasts11-3-2020.xlsx')

#Making the bucket label column into floats (and taking out the " Da" part)
for i in range(len(formula_file)):
    formula_file.iloc[i,0] = np.float(formula_file.iloc[i,0][:-3])
formula_file

307.0838178877 307.0838178877 Da


,label,m/z,Name,Formula,KEGG,BY0_000001,BY0_000002,BY0_000003,GRE3_000001,GRE3_000002,GRE3_000003,ENO1_000001,ENO1_000002,ENO1_000003,dGLO1_000001,dGLO1_000002,dGLO1_000003,GLO2_000001,GLO2_000002,GLO2_000003
0,307.084,308.091097,Glutathione,C10H17N3O6S,C02471,1.482951e+09,1.520914e+09,1.515563e+09,1.231184e+09,1.205245e+09,1.227117e+09,7.151679e+08,7.148845e+08,7.031374e+08,5.903519e+08,5.864704e+08,5.959080e+08,287722656.0,2.914869e+08,2.920921e+08
1,555.269,556.276570,Enkephalin L,C28H37N5O7,NaN,4.219694e+08,4.219694e+08,4.219694e+08,4.219694e+08,4.219694e+08,4.219694e+08,4.219694e+08,4.219694e+08,4.219694e+08,4.219694e+08,4.219694e+08,4.219694e+08,421969440.0,4.219694e+08,4.219694e+08
2,624.087,625.094568,NaN,C13H31F4N2O15P3,NaN,4.933358e+08,4.858229e+08,4.977819e+08,5.939113e+08,5.818669e+08,5.844022e+08,3.827252e+08,3.834892e+08,3.807282e+08,2.709398e+08,2.645076e+08,2.677631e+08,131498984.0,1.330865e+08,1.320918e+08
3,493.317,494.324096,LysoPC_16:1_9Z_0:0_,C24H48NO7P,C04230,4.026932e+08,4.174214e+08,4.232130e+08,4.742500e+08,4.770647e+08,4.894069e+08,8.762351e+07,8.903744e+07,8.863885e+07,8.300221e+07,8.356744e+07,8.238210e+07,56428384.0,5.809368e+07,5.933968e+07
4,257.103,258.110160,Glycerophosphocholine,C8H20NO6P,C00670,2.056732e+08,2.052868e+08,2.117945e+08,2.113430e+08,1.993518e+08,2.069864e+08,2.030151e+08,1.999564e+08,1.930296e+08,1.212009e+08,1.165595e+08,1.169341e+08,59549848.0,6.113070e+07,6.227151e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21169,2029.12,2030.126139,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.487363e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00
21170,532.812,533.819665,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.487326e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00
21171,1070,1071.004232,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.487324e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00
21172,1343.9,1344.907878,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.487058e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00


### Function to assign a formula and after function to "write" the formula in a correct way

In [19]:
def form_checker(mass, threshold, threshppm, dicta):
    #allposs = []
    #i=0
    possible_ma = []
    for x in dicta.itertuples():
        if (mass-threshold) <= x[0] <= (mass+threshold):
            error = ((mass - x[0])/x[0])*1000000
            if abs(error) <= threshppm:
                #ideally enter Senior and Lewis check - next thing to be developed
                possible_ma.append(x)
                #print(x[0], formulator(x[2],x[3],x[4],x[5],x[6],x[7]))
                
                #THINGS FROM THE ORIGINAL FUNCTION THAT I FOUND WERE NOT NEEDED
                
                #allposs.append(list(x[1:9]))
                #allposs[i].append(error)
                #allposs[i].append(intensity)
                #allposs[i].append(mass)
                #formulatemp = FTPM.formulator(int(x[3]),int(x[4]),int(x[6]),int(x[5]),int(x[7]),int(x[8]),int(x[9]),int(x[10]),ionisationmode)
                #dbe = FTPM.DBEcalc(int(x[3]),int(x[4]),int(x[6]),ionisationmode)
                #allposs[i].append(dbe)
                #allposs[i].append(formulatemp)
                #i = i +1
                
    #sENIOR AND LEWIS CHECK SHOULD BE HERE
    if len(possible_ma) > 1:
        min_dif = []
        for i in range(len(possible_ma)):
            min_dif.append(abs(possible_ma[i][0] - mass))
        mini = np.argmin(min_dif)
        return(mass, formulator(possible_ma[mini][2],possible_ma[mini][3],possible_ma[mini][4],possible_ma[mini][5],possible_ma[mini][6],possible_ma[mini][7]))
    elif len(possible_ma) == 1:
        return(mass, formulator(possible_ma[0][2],possible_ma[0][3],possible_ma[0][4],possible_ma[0][5],possible_ma[0][6],possible_ma[0][7]))
    else:
        return(mass, np.nan)

In [20]:
#Adapted from FTMSVisualization
def formulator(c,h,o,n,s,p):
    """Transforms element counts to a readable formula in string format. Element order: C, H, N, O, S and P."""
    formula = "C"+str(c)+"H"+str(h)
    if n > 0:
        if n > 1:
            formula = formula + "N" + str(n)
        else:
            formula = formula + "N"
    if o > 0:
        if o > 1:
            formula = formula + "O" + str(o)
        else:
            formula = formula + "O"
    if s > 0:
        if s > 1:
            formula = formula + "S" + str(s)
        else:
            formula = formula + "S"
    if p > 0:
        if p > 1:
            formula = formula + "P" + str(p)
        else:
            formula = formula + "P"
    return formula

### Parameters for formula attribution (FTMSVisualization)

In [18]:
#Define some parameters for our assignment thresholds.
#Relevant parameters
threshold = 0.005 #Error threshold for formulae in Da - i.e. absolute error threshold.
threshppm = 1.0 # Error threshold for formulae in ppm - i.e. relative error threshold.


#Parameters concerning Isotope finding, checking and the like which isn't needed for this kind of Formula assignment.
#isothresh = 0.0001 # threshold for isotope peak checker #Function not currently active
precisionfactor = 1000#0#0000 # This is a work in progress. THis is used for isotope hunting. A value of 1000 = 1.003355*1000,
#is equal to a 1 mDa error threshold, at 500 m/z this is 0.2 ppm.


#Parameters concerning Kendrick Mass Defect series that they used in a myriad of functions. I don't understand its relevancy.
maxgap = 0.0003 #gap between KMDs for separating series. Units are Da, hence the small numbers. This value will be a compromise, and depend on complexity of your dataset.
# Looking at Z* and KMD method compared to other methods it seems an appropriate range could be 0.0002 or even 0.00002. Look at closer in a future version
minKMDseries = 1 #minimum number of peaks in a single homologous series to assign by zstar approach # 3 seems a good number.

### Test with a small part of the dataset

In [79]:
teste = formula_file[formula_file['label']< 200]
teste

,label,m/z,Name,Formula,KEGG,BY0_000001,BY0_000002,BY0_000003,GRE3_000001,GRE3_000002,GRE3_000003,ENO1_000001,ENO1_000002,ENO1_000003,dGLO1_000001,dGLO1_000002,dGLO1_000003,GLO2_000001,GLO2_000002,GLO2_000003
74,194.115,233.078461,Tetraethylene glycol,C8H18O5,NaN,0.0,0.0,0.0,8.602024e+06,9.371144e+06,8.738521e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0
101,188.105,227.068084,Azelaic acid,C9H16O4,C08261,0.0,0.0,0.0,8.437384e+06,9.330059e+06,9.128233e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.194726e+06,0.0,0.0,0.0
137,187.121,226.084055,N-Heptanoylglycine,C9H17NO3,NaN,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.749687e+06,0.000000e+00,2.265409e+06,2.242338e+06,2.368078e+06,0.0,0.0,0.0
147,186.089,225.052442,5-_2-Methylpropyl_tetrahydro-2-oxo-3-furancarb...,C9H14O4,NaN,0.0,0.0,0.0,4.850851e+06,4.553961e+06,4.521906e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0
172,190.121,229.083634,NaN,C9H18O4,NaN,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.678466e+06,2.134808e+06,1.921604e+06,0.000000e+00,0.000000e+00,1.197077e+06,0.0,0.0,0.0


In [108]:
for i in range(len(teste)):
    mass = teste.iloc[i,0]
    print(form_checker(mass, threshold, threshppm))

194.115418 C7H12N7
188.10486 C9H16O4
187.120844 C9H17N1O3
186.08921 C9H14O4
190.12051 C9H18O4


### Assigning formula to the full dataset

Cut the intensity and samples out of the dataset

In [21]:
formula_file2 = formula_file[['label','m/z','Name','Formula','KEGG']]

In [22]:
#Make a column to store the formulas with each of the labels.
forma = pd.DataFrame(np.zeros((len(formula_file2),1)), index = formula_file2['label'], columns = ['Form_give'])

### Try to assign a formula to each peak on the dataset

In [34]:
for i in range (1,12):
    teste = formula_file[formula_file2['label'] < (i+1)*100]
    teste = teste[teste['label'] > i*100]
    for j in range(len(teste)):
        mass = teste.iloc[j,0]
        tup = form_checker(mass, threshold, threshppm, dicta = formulas['dict' + str(i*100)])
        forma.loc[tup[0]] = tup[1]
        #print(j)
    #print(i*100, 'complete')


### Just until m/z 900 for now

In [55]:
forma.loc[forma.index<900]

,Form_give
label,
307.083818,C10H17N3O6S
555.269298,C28H37N5O7
624.087308,C33H28N2OSP2
493.316816,C21H53N2O4S3
257.102875,NaN
...,...
550.505569,C32H72NOS2
797.227342,C32H41N12O5P2
535.492464,C29H65N3O5


In [27]:
ccccc = forma

In [37]:
formula_file2 = formula_file2.set_index('label')
ending = pd.concat([formula_file2,forma], axis = 1)

In [57]:
ending = ending.loc[ending.index<900]
colus = ending[['Formula','Form_give']]

colus2 = colus.notnull()
print('-----------Number of formulas given by the MetaboScape (Formula) and this notebook (Form_give)---------------')
print(colus2.sum())
print('Total number of peaks:', len(ending))

-----------Number of formulas given by the MetaboScape (Formula) and this notebook (Form_give)---------------
Formula       3872
Form_give    17089
dtype: int64
Total number of peaks: 17334


Almost all peaks had a formula associated not the best

In [65]:
intersection = 0
cls = 0
for i in range(len(colus)):
    if colus.iloc[i,0] == colus.iloc[i,1]:
        intersection = intersection + 1
    if type(colus.iloc[i,0]) == type('str'):
        if 'Cl' in colus.iloc[i,0]:
            cls = cls + 1
        elif 'F' in colus.iloc[i,0]:
            cls = cls + 1
    #print(colus.iloc[i,0], colus.iloc[i,1])
print(intersection, cls, 3872 - cls, intersection/(3872 - cls))

326 2903 969 0.3364293085655315


### Miscellaneous non-relevant tests

In [25]:
#pd.read_table('D/dict100')

In [179]:
teste = formula_file[formula_file['label']< 300]
teste = teste[teste['label']> 200]
teste

,label,m/z,Name,Formula,KEGG
4,257.103,258.110160,Glycerophosphocholine,C8H20NO6P,C00670
7,254.225,255.231904,Hypogeic acid,C16H30O2,NaN
13,256.24,257.247559,Palmitic acid,C16H32O2,C00249
20,243.183,244.190777,N-Undecanoylglycine,C13H25NO3,NaN
22,278.152,317.114972,alpha-CEHC,C16H22O4,NaN
...,...,...,...,...,...
20108,262.23,263.236853,"6,10,14-Trimethyl-5,9,13-pentadecatrien-2-one",C18H30O,NaN
20521,226.121,227.127818,Allixin,C12H18O4,NaN
20770,269.272,270.279028,Capsiamide,C17H35NO,C17515
20991,266.95,267.957701,NaN,NaN,NaN


In [182]:
forma = pd.DataFrame(np.zeros((len(teste),1)), index = teste['label'], columns = ['Form_give'])
#print(forma)
for i in range(len(teste)):
    mass = teste.iloc[i,0]
    tup = form_checker(mass, threshold, threshppm, dicta = formulas['dict200'])
    forma.loc[tup[0]] = tup[1]
print(forma)


             Form_give
label                 
257.102875         NaN
254.224610    C16H30O2
256.240207    C16H32O2
243.183406   C13H25NO3
278.151833    C16H22O4
...                ...
262.229577     C18H30O
226.120541    C12H18O4
269.271752    C17H35NO
266.950424  C4H5N4O4SP
256.239165         NaN

[420 rows x 1 columns]


In [169]:
#pd.concat([teste.set_index('label'),forma])
teste = teste.set_index('label')
pd.concat([teste,forma], axis = 1)
#forma

,m/z,Name,Formula,KEGG,BY0_000001,BY0_000002,BY0_000003,GRE3_000001,GRE3_000002,GRE3_000003,ENO1_000001,ENO1_000002,ENO1_000003,dGLO1_000001,dGLO1_000002,dGLO1_000003,GLO2_000001,GLO2_000002,GLO2_000003,Form_give
label,,,,,,,,,,,,,,,,,,,,
257.102875,258.110160,Glycerophosphocholine,C8H20NO6P,C00670,2.056732e+08,2.052868e+08,2.117945e+08,2.113430e+08,1.993518e+08,2.069864e+08,2.030151e+08,1.999564e+08,1.930296e+08,1.212009e+08,1.165595e+08,1.169341e+08,59549848.0,6.113070e+07,6.227151e+07,NaN
254.224610,255.231904,Hypogeic acid,C16H30O2,NaN,3.768004e+07,6.755935e+07,7.409344e+07,2.596554e+07,4.509098e+07,7.337231e+07,3.014258e+07,7.357830e+07,8.325845e+07,5.520110e+07,5.732034e+07,5.935389e+07,9995730.0,1.379522e+07,1.950888e+07,C16H30O2
256.240207,257.247559,Palmitic acid,C16H32O2,C00249,4.938765e+07,5.654261e+07,5.656344e+07,5.135225e+07,4.646735e+07,4.946956e+07,3.967435e+07,5.034116e+07,5.555791e+07,3.700789e+07,3.896814e+07,4.062987e+07,13851696.0,1.279689e+07,1.322522e+07,C16H32O2
243.183406,244.190777,N-Undecanoylglycine,C13H25NO3,NaN,1.923931e+07,1.813270e+07,1.958480e+07,2.611610e+07,2.529480e+07,2.436860e+07,1.036602e+07,1.101832e+07,1.081115e+07,1.092722e+07,1.230570e+07,1.234905e+07,5295285.5,5.204488e+06,4.870206e+06,C13H25NO3
278.151833,317.114972,alpha-CEHC,C16H22O4,NaN,1.389726e+07,1.454467e+07,1.619012e+07,1.631480e+07,1.342425e+07,1.304407e+07,1.132682e+07,1.062885e+07,1.030479e+07,6.430070e+06,6.741803e+06,5.763232e+06,5298218.5,4.351453e+06,4.133799e+06,C16H22O4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262.229577,263.236853,"6,10,14-Trimethyl-5,9,13-pentadecatrien-2-one",C18H30O,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.111047e+06,0.0,0.000000e+00,0.000000e+00,C18H30O
226.120541,227.127818,Allixin,C12H18O4,NaN,0.000000e+00,0.000000e+00,0.000000e+00,2.487062e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,C12H18O4
269.271752,270.279028,Capsiamide,C17H35NO,C17515,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.096925e+06,0.0,0.000000e+00,0.000000e+00,C17H35NO


In [132]:
dict23.loc[dict23['C'] == 16].loc[dict23['H'] == 32]
teste.iloc[-1,0]

256.2391650817

In [133]:
256.240230
((256.2391650817 - 256.240230)/256.240230)*1000000

-4.155937184480636

In [110]:
dict23 = pd.DataFrame.from_dict(a, orient = 'index', columns = ['Abundance', 'C','H','O','N','S','P'])
#dict12.to_csv('Formulas100-200.csv')
dict23 = dict23.sort_index()
print(dict23)

            Abundance   C   H   O  N  S  P
200.000412   0.878951   5   4   3  4  1  0
200.001218   0.852422   5  14   0  0  2  1
200.001687   0.928933   5   6   1  4  0  1
200.001755   0.866743   7   6   4  1  1  0
200.003030   0.916031   7   8   2  1  0  1
...               ...  ..  ..  .. .. .. ..
299.999174   0.869468  10   6  10  1  0  0
299.999349   0.776458  17   4   2  2  1  0
299.999504   0.772499   9  18   1  0  3  1
299.999912   0.903729   8  16   4  0  0  2
299.999973   0.841835   9  10   2  4  1  1

[59074 rows x 7 columns]


In [56]:
dict12 = pd.DataFrame.from_dict(a, orient = 'index', columns = ['Abundance', 'C','H','O','N','S','P'])
#dict12.to_csv('Formulas100-200.csv')
dict12 = dict12.sort_index()
print(dict12)

            Abundance  C   H  O  N  S  P
100.003469   0.956417  3   2  3  1  0  0
100.009519   0.911867  3   4  0  2  1  0
100.016045   0.948991  4   4  3  0  0  0
100.018724   0.921291  7   2  0  1  0  0
100.022095   0.904787  4   6  0  1  1  0
...               ... ..  .. .. .. .. ..
199.995229   0.832837  6   6  1  3  2  0
199.995705   0.909089  7   4  7  0  0  0
199.996572   0.821269  8   8  2  0  2  0
199.997847   0.867971  8  10  0  0  1  1
199.999943   0.806557  5  12  2  0  3  0

[9103 rows x 7 columns]


In [84]:
dicta = dict12
for x in dicta.itertuples():
    print(x)

Pandas(Index=100.003469, Abundance=0.956417475735444, C=3, H=2, O=3, N=1, S=0, P=0)
Pandas(Index=100.009519, Abundance=0.9118672949844809, C=3, H=4, O=0, N=2, S=1, P=0)
Pandas(Index=100.016045, Abundance=0.9489910390016091, C=4, H=4, O=3, N=0, S=0, P=0)
Pandas(Index=100.01872399999999, Abundance=0.9212907319021914, C=7, H=2, O=0, N=1, S=0, P=0)
Pandas(Index=100.02209500000001, Abundance=0.9047867836516574, C=4, H=6, O=0, N=1, S=1, P=0)
Pandas(Index=100.027278, Abundance=0.9548847043291426, C=3, H=4, O=2, N=2, S=0, P=0)
Pandas(Index=100.0313, Abundance=0.9141370490100225, C=8, H=4, O=0, N=0, S=0, P=0)
Pandas(Index=100.034671, Abundance=0.8977612514161323, C=5, H=8, O=0, N=0, S=1, P=0)
Pandas(Index=100.039854, Abundance=0.947470169332954, C=4, H=6, O=2, N=1, S=0, P=0)
Pandas(Index=100.051087, Abundance=0.9533543893691564, C=3, H=6, O=1, N=3, S=0, P=0)
Pandas(Index=100.05243, Abundance=0.9401132070771814, C=5, H=8, O=2, N=0, S=0, P=0)
Pandas(Index=100.063663, Abundance=0.9459517370366806,

Pandas(Index=159.017641, Abundance=0.8405645840917407, C=6, H=9, O=0, N=1, S=2, P=0)
Pandas(Index=159.019453, Abundance=0.9032879783246471, C=8, H=3, O=2, N=2, S=0, P=0)
Pandas(Index=159.022824, Abundance=0.8871065303480468, C=5, H=7, O=2, N=2, S=1, P=0)
Pandas(Index=159.023475, Abundance=0.8647421025473863, C=13, H=3, O=0, N=0, S=0, P=0)
Pandas(Index=159.02409899999998, Abundance=0.9375522909480841, C=5, H=9, O=0, N=2, S=0, P=1)
Pandas(Index=159.026846, Abundance=0.8492511631334694, C=10, H=7, O=0, N=0, S=1, P=0)
Pandas(Index=159.028007, Abundance=0.9362254978140502, C=4, H=5, O=4, N=3, S=0, P=0)
Pandas(Index=159.02935000000002, Abundance=0.9232222194726624, C=6, H=7, O=5, N=0, S=0, P=0)
Pandas(Index=159.030217, Abundance=0.8340377275015688, C=7, H=11, O=0, N=0, S=2, P=0)
Pandas(Index=159.030686, Abundance=0.9088978068897626, C=7, H=3, O=1, N=4, S=0, P=0)
Pandas(Index=159.032029, Abundance=0.8962740840853107, C=9, H=5, O=2, N=1, S=0, P=0)
Pandas(Index=159.034057, Abundance=0.892615864

Pandas(Index=179.146961, Abundance=0.8545318503550704, C=9, H=23, O=1, N=0, S=1, P=0)
Pandas(Index=179.152144, Abundance=0.9018471622373834, C=8, H=21, O=3, N=1, S=0, P=0)
Pandas(Index=179.154823, Abundance=0.8755229480730474, C=11, H=19, O=0, N=2, S=0, P=0)
Pandas(Index=179.158194, Abundance=0.8598388812233589, C=8, H=23, O=0, N=2, S=1, P=0)
Pandas(Index=179.163377, Abundance=0.9074480426802639, C=7, H=21, O=2, N=3, S=0, P=0)
Pandas(Index=179.16472000000002, Abundance=0.8948444557165784, C=9, H=23, O=3, N=0, S=0, P=0)
Pandas(Index=179.167399, Abundance=0.8687246450852384, C=12, H=21, O=0, N=1, S=0, P=0)
Pandas(Index=179.17077, Abundance=0.8531623626373864, C=9, H=25, O=0, N=1, S=1, P=0)
Pandas(Index=179.175953, Abundance=0.9004018461717517, C=8, H=23, O=2, N=2, S=0, P=0)
Pandas(Index=179.179975, Abundance=0.8619791298897036, C=13, H=23, O=0, N=0, S=0, P=0)
Pandas(Index=179.183346, Abundance=0.8465376862062667, C=10, H=27, O=0, N=0, S=1, P=0)
Pandas(Index=179.18852900000002, Abundance=

Pandas(Index=194.01232, Abundance=0.8838817336615232, C=5, H=8, O=5, N=1, S=1, P=0)
Pandas(Index=194.01318700000002, Abundance=0.7984975848439627, C=6, H=12, O=0, N=1, S=3, P=0)
Pandas(Index=194.01359499999998, Abundance=0.9341441145703206, C=5, H=10, O=3, N=1, S=0, P=1)
Pandas(Index=194.013656, Abundance=0.8701677151290312, C=6, H=4, O=1, N=5, S=1, P=0)
Pandas(Index=194.014999, Abundance=0.8580819163231549, C=8, H=6, O=2, N=2, S=1, P=0)
Pandas(Index=194.01627399999998, Abundance=0.9068771775969909, C=8, H=8, O=0, N=2, S=0, P=1)
Pandas(Index=194.01837, Abundance=0.8427102870954554, C=5, H=10, O=2, N=2, S=2, P=0)
Pandas(Index=194.018839, Abundance=0.9183487827090663, C=5, H=2, O=3, N=6, S=0, P=0)
Pandas(Index=194.019021, Abundance=0.8214651122175003, C=13, H=6, O=0, N=0, S=1, P=0)
Pandas(Index=194.01964499999997, Abundance=0.8906314329146923, C=5, H=12, O=0, N=2, S=1, P=1)
Pandas(Index=194.020182, Abundance=0.9055937948734193, C=7, H=4, O=4, N=3, S=0, P=0)
Pandas(Index=194.021525, Abund

In [19]:
a = 0

In [50]:
com_range['H/C'][0]

0.2

In [44]:
#if not ((n < 11) and (o < 22) and (p < 6)):
#    print('HELLO')

HELLO


In [13]:
#hi = '34s'
#hi = hi + 'sd'
#hi

'34ssd'

In [16]:
#n, o, p, s = 12,20,5,20
#import numpy as np

In [32]:
NOPS_ratio = True
if (n > 1) and (o > 1) and (p > 1) and (s > 1): #NOPS
    if (n < 10) and (o < 20) and (p < 4) and (s < 3):
        NOPS_ratio = True
    else:
        NOPS_ratio = False
elif (n > 3) and (o > 3) and (p > 3): #NOP
    if (n < 11) and (o < 22) and (p < 6):
        NOPS_ratio = True
    else:
        NOPS_ratio = False
elif (o > 1) and (p > 1) and (s > 1): #OPS
    if (o < 14) and (p < 3) and (s < 3):
        NOPS_ratio = True
    else:
        NOPS_ratio = False
elif (n > 1) and (p > 1) and (s > 1): #PSN
    if (n < 10) and (p < 4) and (s < 3):
        NOPS_ratio = True
    else:
        NOPS_ratio = False
elif (n > 6) and (o > 6) and (s > 6): #PSN
    if (n < 19) and (o < 14) and (s < 8):
        NOPS_ratio = True
    else:
        NOPS_ratio = False
NOPS_ratio

False